## GDAL with HDF5 and VRT

In [ ]:
from common import earthaccess_args
from osgeo import gdal
from pyproj.crs import CRS

In [ ]:
gdal.UseExceptions()

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
variable = earthaccess_args[dataset]["variable"]
a_ullr = earthaccess_args[dataset]["a_ullr"]

In [ ]:
def warp_resample():
    src = f"vrt://earthaccess_data/{filename}?a_ullr={a_ullr}&sd_name={variable}"
    output = ""
    output_format = "MEM"
    dstSRS = "EPSG:3857"

    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]
    gt = [
        te[0],
        (te[2] - te[0]) / width,
        0,
        te[3],
        0,
        -(te[3] - te[1]) / height,
    ]
    output_crs = CRS(dstSRS).to_wkt()

    output_ds = gdal.GetDriverByName(output_format).Create(
        output, width, height, 1, gdal.GDT_Byte
    )
    output_ds.SetProjection(output_crs)
    output_ds.SetGeoTransform(gt)
    input_ds = gdal.Open(src)
    return gdal.Warp(output_ds, input_ds)

In [ ]:
if __name__ == "__main__":
    arr = warp_resample()